In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import reverse_geocoder as rg

# read and preprocess data
try:
    for i in range(28):
        df = pd.read_csv(('input_data_raw/{}.csv.gz').format(i+1),sep=',',encoding='utf-8')
        df.drop(labels=['number','icao24','registration','typecode','firstseen','lastseen','altitude_1','altitude_2'], 
                axis=1,
                inplace=True)
        df['ICAO'] = df['callsign'].apply(lambda x: x[0:3])

        # table airline-icao-code
        df_icao_code = pd.read_excel('aux_files/IATA_airlines_code.xlsx',sheet_name='airline_code',)
        df_icao_code.drop(['Call sign','Comments'],inplace=True,axis=1)
        df_icao_code.columns = ['ICAO','Airline','Airline_country']

        # merge ICAO table
        df_m = df.merge(df_icao_code, how='inner', on='ICAO')
        df_m.drop('callsign',axis=1,inplace=True)

        # table airport-codes
        air = pd.read_csv('aux_files/airports.csv',encoding='utf-8',sep=';',keep_default_na=False)
        air = air.drop(['type','name','iso_region'],axis=1)

        # merge airport table
            ## merge origin
        df_m2 = pd.merge(left=df_m,right=air,how='left',left_on='origin',right_on='ident')
        df_m2 = df_m2.drop(['ident'],axis=1)
        df_m2.columns = ['origin', 'destination', 'day', 'latitude_1', 'longitude_1',
                         'latitude_2', 'longitude_2', 'ICAO', 'Airline', 'Airline_country',
                         'continent_o', 'iso_country_o', 'municipality_o']
            ## merge destination
        df_m2 = pd.merge(left=df_m2,right=air,how='left',left_on='destination',right_on='ident')
        df_m2 = df_m2.drop(['ident'],axis=1)
        df_m2.columns = ['origin', 'destination', 'day', 'latitude_1', 'longitude_1',
                         'latitude_2', 'longitude_2', 'ICAO', 'Airline', 'Airline_country',
                         'continent_o', 'iso_country_o', 'municipality_o', 
                         'continent_d','iso_country_d', 'municipality_d']

        # day column to datetime type
        df_m2['day'] =  df_m2['day'].apply(lambda x: x[0:11])
        df_m2['day'] = pd.to_datetime(df_m2['day'])


        # to locations from coordenates
        df_clean = df_m2.copy()
            ## delete rows without information about location
        rows_o_del = df_m2[df_m2['origin'].isnull() & (df_m2['latitude_1'].isnull() | df_m2['longitude_1'].isnull())].index
        rows_d_del = df_m2[df_m2['destination'].isnull() & (df_m2['latitude_2'].isnull() | df_m2['longitude_2'].isnull())].index
        df_clean.drop(rows_o_del, inplace=True)
        df_clean.drop(rows_d_del, inplace=True)
        df_clean.reset_index(drop=True, inplace=True)

            ## fill missing coordenates
        df_clean['latitude_1'].fillna(-90, inplace=True)
        df_clean['latitude_2'].fillna(-90, inplace=True)
        df_clean['longitude_1'].fillna(0, inplace=True)
        df_clean['longitude_2'].fillna(0, inplace=True)

            ## dataframe with locations from coordenates
        coordenates_o = list(zip(df_clean['latitude_1'],df_clean['longitude_1']))
        coordenates_d = list(zip(df_clean['latitude_2'],df_clean['longitude_2']))
        coor_o_res = rg.search(coordenates_o)
        coor_d_res = rg.search(coordenates_d)

        o_city = []
        o_region = []
        o_iso = []
        d_city = []
        d_region = []
        d_iso = []

        for loc in range(len(coor_o_res)):
            o_city.append(list(coor_o_res[loc].values())[2])
            o_region.append(list(coor_o_res[loc].values())[3])
            o_iso.append(list(coor_o_res[loc].values())[5])

            d_city.append(list(coor_d_res[loc].values())[2])
            d_region.append(list(coor_d_res[loc].values())[3])
            d_iso.append(list(coor_d_res[loc].values())[5])

            ## concat main df and df-locations
        df_locations = pd.DataFrame({'o_city':o_city, 'o_region':o_region, 'o_iso':o_iso, 'd_city':d_city, 'd_region':d_region, 'd_iso':d_iso})

            ## delete false locations from origin
        df_locations['o_city'].mask((df_locations['o_iso']=='GS'), np.nan, inplace=True)
        df_locations['o_region'].mask((df_locations['o_iso']=='GS'), np.nan, inplace=True)
        df_locations['o_iso'].mask((df_locations['o_iso']=='GS'), np.nan, inplace=True)

            ## delete false locations from destination
        df_locations['d_city'].mask((df_locations['d_iso']=='GS'), np.nan, inplace=True)
        df_locations['d_region'].mask((df_locations['d_iso']=='GS'), np.nan, inplace=True)
        df_locations['d_iso'].mask((df_locations['d_iso']=='GS'), np.nan, inplace=True)

        df_clean2 = pd.concat([df_clean,df_locations],axis=1)

            ## delete rows with same origin and destination
        rows_cities_del = df_clean2[df_clean2['o_city']==df_clean2['d_city']].index
        df_clean2.drop(rows_cities_del, inplace=True)
        df_clean2.reset_index(drop=True,inplace=True)

        # unify locations
        df_final = df_clean2.copy()

        df_final['origin_city'] = df_final.apply(get_origin_city, axis=1)
        df_final['destination_city'] = df_final.apply(get_destination_city, axis=1)
        df_final['origin_country'] = df_final.apply(get_origin_country, axis=1)
        df_final['destination_country'] = df_final.apply(get_destination_country, axis=1)


        # final data frame
        df_final_2 = df_final.copy()
        df_final_2.drop(['origin', 'destination','ICAO','continent_o', 'iso_country_o', 'municipality_o', 'continent_d', 
                       'iso_country_d', 'municipality_d', 'o_city', 'o_iso','d_city','d_iso'],axis=1,inplace=True)
        df_final_2.columns = ['day', 'latitude_1', 'longitude_1', 'latitude_2', 'longitude_2',
                            'Airline', 'Airline_country', 'origin_region', 'destination_region', 
                            'origin_city','destination_city', 'origin_country', 'destination_country']

        # translate country code to name
            ## table with code country, continent and name
        df_country_cont = pd.read_csv('aux_files/country-and-continent-codes-list-csv.csv',sep=';',keep_default_na=False)

            ## merge with origins
        df_final_2 = pd.merge(left=df_final_2, right=df_country_cont, how='left', 
                              left_on='origin_country',right_on='Two_Letter_Country_Code')
        df_final_2.drop(['Two_Letter_Country_Code','origin_country'], axis=1, inplace=True)
        df_final_2.columns = ['day', 'latitude_1', 'longitude_1', 'latitude_2', 'longitude_2',
                              'Airline', 'Airline_country', 'origin_region', 'destination_region',
                              'origin_city', 'destination_city', 'destination_country',
                              'origin_continent', 'origin_country']

            ## merge with destinations
        df_final_2 = pd.merge(left=df_final_2, right=df_country_cont, how='left', 
                              left_on='destination_country',right_on='Two_Letter_Country_Code')
        df_final_2.drop(['Two_Letter_Country_Code','destination_country'], axis=1, inplace=True)
        df_final_2.columns = ['date', 'latitude_1', 'longitude_1', 'latitude_2', 'longitude_2',
                              'airline', 'airline_country', 'origin_region', 'destination_region',
                              'origin_city', 'destination_city', 'origin_continent', 'origin_country',
                              'destination_continent', 'destination_country']


        # back to original coordenates
        df_final_3 = df_final_2.copy()
        df_final_3['latitude_1'].mask((df_final_3['latitude_1']==-90) & (df_final_3['longitude_1']==0), np.nan, inplace=True)
        df_final_3['longitude_1'].mask(df_final_3['latitude_1'].isna(), np.nan, inplace=True)
        df_final_3['latitude_2'].mask((df_final_3['latitude_2']==-90) & (df_final_3['longitude_2']==0), np.nan, inplace=True)
        df_final_3['longitude_2'].mask(df_final_3['latitude_2'].isna(), np.nan, inplace=True)

        # fill void strings with nulls
        df_final_3.replace({'': np.nan}, inplace=True)

        #write result dataframe to csv
        df_final_3.to_csv(('input_data_clean/{}_clean.csv.gz').format(i+1), sep=',', index=False,encoding='utf-8')
        print(('DONE {}').format(i+1))
        
except:
    print(('FAIL {}').format(i+1))

C:\Users\victo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Loading formatted geocoded file...
FAIL 1


In [2]:
# functions to unify locations

def get_origin_city(df):
    if pd.isna(df['origin']):
        return df['o_city']
    elif pd.isna(df['municipality_o']):
        return df['o_city']
    else:
        return df['municipality_o']
    
def get_destination_city(df):
    if pd.isna(df['destination']):
        return df['d_city']
    elif pd.isna(df['municipality_d']):
        return df['d_city']
    else:
        return df['municipality_d']
    
def get_origin_country(df):
    if pd.isna(df['origin']):
        return df['o_iso']
    elif pd.isna(df['iso_country_o']):
        return df['o_iso']
    else:
        return df['iso_country_o']
    
def get_destination_country(df):
    if pd.isna(df['destination']):
        return df['d_iso']
    elif pd.isna(df['iso_country_d']):
        return df['d_iso']
    else:
        return df['iso_country_d']